<img src="report_material/feup_logo_old.png" width="30%" align="center"/>

<h1 style="text-align:center">Neural Networks for the Prediction of Anuran Species (E3.)</h1>
<h3 style="text-align:center">Intermediate Report</h3>
<br/><br/>
<p style="text-align:center">Artificial Intelligence (IART)</p>
<p style="text-align:center">3<sup>rd</sup> year in Master in Informatics and Computing Engineering</p>

### Authors

* Miguel Ramalho    - 201403027 - [m.ramalho@fe.up.pt](mailto:up201403027@fe.up.pt)
* Rostyslav Khoptiy - 201506219 - [up201506219@fe.up.pt](mailto:up201506219@fe.up.pt)

<!--spellchecking: https://stackoverflow.com/a/48710561/6196010 -->

<div style="text-align:right"><i>April 8<sup>th</sup>, 2018</i></div>
<br/><br/>

---

# IART

* [GitHub repository](https://github.com/msramalho/iart)
* [UCI source page](http://archive.ics.uci.edu/ml/datasets/Anuran+Calls+%28MFCCs%29)
* [README](dataset/README.ipynb) of the dataset
* [Dataset file](dataset/frogs.csv)
* [Automatic Classification of Anuran Sounds Using Convolutional Neural Networks](report_material/papers/Automatic%20Classification%20of%20Anuran%20Sounds%20Using.pdf)
* [Tensorflow tutorial](https://www.tensorflow.org/get_started/premade_estimators)

# Code

## Initialization

In [2]:
import pandas as pd
import tensorflow as tf
import io # to pass data as StringIO object
import time # to set the random seed for train and test sampling
import matplotlib.pyplot as plt
from sklearn.model_selection import KFold # for crossvalidation, needs scipy

In [3]:
DATASET_PATH = "dataset/frogs.csv" # the path to the dataset file
N_FEATURES = 22 # the number of features in the dataset
LABEL_INDEX = 24 # the index in the dataset where the label to predict is
BATCH_SIZE = 100 # batch size for the train, evaluation and prediction
NUM_CLASSES = 10 # last layer, num classifications
NUM_ITEMS = 7195 #Num items in dataset

## Helper Functions

In this section we include functions that are directed at helping the visualisation of the project, the manipulation of data and the creation, training and evaluation of models.

### Chart Drawing functions

In [5]:
def draw_neural_net(ax, left, right, bottom, top, layer_sizes):
    # https://gist.github.com/craffel/2d727968c3aaebd10359
    n_layers = len(layer_sizes)
    v_spacing = (top - bottom)/float(max(layer_sizes))
    h_spacing = (right - left)/float(max(len(layer_sizes) - 1, 1))
    # Nodes
    for n, layer_size in enumerate(layer_sizes):
        layer_top = v_spacing*(layer_size - 1)/2. + (top + bottom)/2.
        for m in range(layer_size):
            circle = plt.Circle((n*h_spacing + left, layer_top - m*v_spacing), v_spacing/4.,
                                color='w', ec='k', zorder=4)
            ax.add_artist(circle)
    # Edges
    for n, (layer_size_a, layer_size_b) in enumerate(zip(layer_sizes[:-1], layer_sizes[1:])):
        layer_top_a = v_spacing*(layer_size_a - 1)/2. + (top + bottom)/2.
        layer_top_b = v_spacing*(layer_size_b - 1)/2. + (top + bottom)/2.
        for m in range(layer_size_a):
            for o in range(layer_size_b):
                line = plt.Line2D([n*h_spacing + left, (n + 1)*h_spacing + left],
                                  [layer_top_a - m*v_spacing, layer_top_b - o*v_spacing], c='k')
                ax.add_artist(line)

def drawNN(hidden_layers, figsize=(12, 12)):
    fig = plt.figure(figsize=figsize)
    ax = fig.gca()
    ax.axis('off')
    draw_neural_net(ax, .1, .9, .1, .9, hidden_layers)
#     fig.savefig('nn.png')
    

In [6]:
def draw_label_distribution(label_column, title_text):
    label_count = pd.Series.value_counts(label_column, normalize=True, sort=False)
    label_count = label_count.sort_index()
    label_count.plot(kind='barh', title=title_text, stacked=True, fontsize=15, figsize=(8,8), color="darkblue" )
    
def draw_species_distribution_chart(df, title_text="Species Dataset Distribution"):
    species_column = df['Species']
    draw_label_distribution(species_column, title_text)

### Creating Models

In [9]:
def create_DNN(my_feature_columns, inner_layers_architecture, save_folder_path=None, num_classes=NUM_CLASSES):
    # Build a DNN (Deep Neural Network) with 1 hidden layer with 10 nodes
    classifier = tf.estimator.DNNClassifier(
            feature_columns = my_feature_columns,
            hidden_units = inner_layers_architecture, # 1:-1 removes the first and last elements of the architecture
            n_classes = num_classes, # num nodes of final layer
            model_dir = save_folder_path) # value None means it will be saved in temp, 
                                            # no need to generate path for each iteration
    return classifier

### Training Models

In [10]:
SHUFFLE_PARAM = NUM_ITEMS * 10 #df.shape[0] * 10
def _train_input_fn(features, labels):
    # Convert the inputs to a Dataset.
    dataset = tf.data.Dataset.from_tensor_slices((dict(features), labels))
    
    # Shuffle, repeat, and batch the examples. The shuffle argument should be larger than the number of examples
    return dataset.shuffle(SHUFFLE_PARAM).repeat().batch(BATCH_SIZE)

In [11]:
def train_model(classifier, train_x, numeric_train_y, num_steps=1000):
    # Train the model - 10 steps
    classifier.train(input_fn=lambda:_train_input_fn(train_x, numeric_train_y), steps=num_steps)

### Evaluating Models

In [12]:
def _eval_input_fn(features, labels=None):
    inputs = features if labels is None else (features, labels)
    
    # Convert inputs to a tf.dataset object. Batch the examples.
    dataset = tf.data.Dataset.from_tensor_slices(inputs).batch(BATCH_SIZE)

    # Return the read end of the pipeline.
    return dataset.make_one_shot_iterator().get_next()

In [13]:
def evaluate_model(classifier, test_x, numeric_test_y):
    # Evaluate the model.
    eval_result = classifier.evaluate(input_fn=lambda:_eval_input_fn(dict(test_x), numeric_test_y))
    return eval_result

## "Main"

### Loading Data

Got 7195 examples from the .csv


,MFCCs_1,MFCCs_2,MFCCs_3,MFCCs_4,MFCCs_5,MFCCs_6,MFCCs_7,MFCCs_8,MFCCs_9,MFCCs_10,...,MFCCs_17,MFCCs_18,MFCCs_19,MFCCs_20,MFCCs_21,MFCCs_22,Family,Genus,Species,RecordID
0,1.0,0.152936,-0.105586,0.200722,0.317201,0.260764,0.100945,-0.150063,-0.171128,0.124676,...,-0.108351,-0.077623,-0.009568,0.057684,0.118680,0.014038,Leptodactylidae,Adenomera,AdenomeraAndre,1
1,1.0,0.171534,-0.098975,0.268425,0.338672,0.268353,0.060835,-0.222475,-0.207693,0.170883,...,-0.090974,-0.056510,-0.035303,0.020140,0.082263,0.029056,Leptodactylidae,Adenomera,AdenomeraAndre,1
2,1.0,0.152317,-0.082973,0.287128,0.276014,0.189867,0.008714,-0.242234,-0.219153,0.232538,...,-0.050691,-0.023590,-0.066722,-0.025083,0.099108,0.077162,Leptodactylidae,Adenomera,AdenomeraAndre,1
3,1.0,0.224392,0.118985,0.329432,0.372088,0.361005,0.015501,-0.194347,-0.098181,0.270375,...,-0.136009,-0.177037,-0.130498,-0.054766,-0.018691,0.023954,Leptodactylidae,Adenomera,AdenomeraAndre,1
4,1.0,0.087817,-0.068345,0.306967,0.330923,0.249144,0.006884,-0.265423,-0.172700,0.266434,...,-0.048885,-0.053074,-0.088550,-0.031346,0.108610,0.079244,Leptodactylidae,Adenomera,AdenomeraAndre,1


In [14]:
# from csv to pandas Dataframe
df = pd.read_csv(DATASET_PATH)

# draw dataset distribution
draw_species_distribution_chart(df)

df.columns = df.columns.str.replace(" ", "") # remove whitespace, needed for TF
print("Got %d examples from the .csv" % df.shape[0])
df.head()

### Dataset Splitting

In [7]:
# converts a list of strings into a list of IDs (numerical values) used for the classifier
def _convert_labels_to_number(labels):
    unique_labels = list(set(labels)) # list of unique values
    return unique_labels, [unique_labels.index(l) for l in labels]

In [37]:
def _format_TF_data(train, test):
    # split dataframes into X (features) and y (classes) - use species as label
    train_x, train_y = train.iloc[:,:N_FEATURES], train.iloc[:,LABEL_INDEX:LABEL_INDEX+1]
    test_x, test_y = test.iloc[:,:N_FEATURES], test.iloc[:,LABEL_INDEX:LABEL_INDEX+1]
    # Convert the features MFCCs_1 to MFCCs_22 into Tensorflow feature columns
    my_feature_columns = [tf.feature_column.numeric_column(key=key) for key in list(train_x)]
    
    # for the 10 species, get a numerical mapping of the text name of that species (needed for DNN)
    _, numeric_train_y = _convert_labels_to_number(train_y.iloc[:,0])
    _, numeric_test_y  = _convert_labels_to_number(test_y.iloc[:,0])
    
    return (train_x, numeric_train_y, test_x, numeric_test_y, my_feature_columns)

In [38]:
def split_data(df, train_fraction):
    # split into 80% train and 20% test, may need crossvalidation in the future
    train=df.sample(frac=train_fraction, random_state=int(time.time()))
    test=df.drop(train.index)

    return _format_TF_data(train, test)

In [48]:
def split_data_by_indices(train_indices, test_indices):
    train = df.iloc[train_indices]
    test = df.iloc[test_indices]
    return _format_TF_data(train, test)
    

In [63]:
def get_k_fold_CV_split_indices(df, num_folds = 3):
    k_fold = KFold(n_splits = num_folds, shuffle=True)
    return k_fold.split(df)
    

### Running iterations

In [82]:
### Iterations
INNER_LAYERS = [
    [10],
    [100],
    [20, 20],
    [100, 100],
#     [10, 10, 10],
]

TRAIN_FRACTIONS = [
    0.8,
#     0.5,
]

NUM_STEPS = [
    100, 
#     1000,
]

In [83]:
# this may take a while
evaluations = []

for inner_architecture in INNER_LAYERS:
    for num_steps in NUM_STEPS:
        for train_fraction in TRAIN_FRACTIONS:
            # data splitting
            (train_x, numeric_train_y, test_x, numeric_test_y, my_feature_columns) = split_data(df, train_fraction)
            
            #drawNN([N_FEATURES] + inner_architecture + [NUM_CLASSES])
            # DNN model
            classifier = create_DNN(my_feature_columns, inner_architecture)
            train_model(classifier, train_x, numeric_train_y, num_steps)
            eval_result = evaluate_model(classifier, test_x, numeric_test_y)
            # evaluation results
            result_tup = ( (inner_architecture, train_fraction, num_steps), eval_result )
            evaluations.append(result_tup)
    


INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': 'C:\\Users\\Ross\\AppData\\Local\\Temp\\tmpa1nic3bw', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x0000026881F82748>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 1 into C:\Users\Ross\AppData\Local\Temp\tmpa1nic3bw\m

In [84]:
# purely temporary
def show_results(evaluations):
    # show results, statistics
    print('\n\n\tNum Results: %d' % len(evaluations) )
    print('>> (inner_arch, train_frac, num_steps) , Accuracy' )
    for ( iteration_tup, eval_result ) in evaluations:
        print(iteration_tup, ',', eval_result['accuracy'])
    #     print('\nTest set accuracy: {accuracy:0.3f}\n'.format(**eval_result))
    
show_results(evaluations)




	Num Results: 4
>> (inner_arch, train_frac, num_steps) , Accuracy
([10], 0.8, 100) , 0.81584436
([100], 0.8, 100) , 0.92911744
([20, 20], 0.8, 100) , 0.9200834
([100, 100], 0.8, 100) , 0.9596942


In [85]:
# k-fold cross validation

kfcv_evaluations = []

for inner_architecture in INNER_LAYERS:
    for num_steps in NUM_STEPS:
        for train_fraction in TRAIN_FRACTIONS:
            classifier = create_DNN(my_feature_columns, inner_architecture)
            num_folds = int(1.0 / (1.0 - train_fraction))
            num_steps /= num_folds # num_total_model_steps = num_steps * num_folds
            for train_indices, test_indices in get_k_fold_CV_split_indices(df, num_folds):
                (train_x, numeric_train_y, test_x, numeric_test_y, my_feature_columns) = split_data_by_indices(train_indices, test_indices)
                train_model(classifier, train_x, numeric_train_y, num_steps)
                eval_result = evaluate_model(classifier, test_x, numeric_test_y)
                # evaluation results
                result_tup = ( (inner_architecture, train_fraction, num_steps), eval_result )
                kfcv_evaluations.append(result_tup)
    
    



INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': 'C:\\Users\\Ross\\AppData\\Local\\Temp\\tmpfqi4tlxk', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x000002688376C438>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 1 into C:\Users\Ross\AppData\Local\Temp\tmpfqi4tlxk\m

INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Finished evaluation at 2018-04-22-12:52:27
INFO:tensorflow:Saving dict for global step 20: accuracy = 0.88255733, average_loss = 0.47432613, global_step = 20, loss = 45.503685
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from C:\Users\Ross\AppData\Local\Temp\tmp25a23_8b\model.ckpt-20
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 21 into C:\Users\Ross\AppData\Local\Temp\tmp25a23_8b\model.ckpt.
INFO:tensorflow:loss = 47.921543, step = 21
INFO:tensorflow:Saving checkpoints for 40 into C:\Users\Ross\AppData\Local\Temp\tmp25a23_8b\model.ckpt.
INFO:tensorflow:Loss for final step: 32.457096.
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2018-04-22-12:

INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 41 into C:\Users\Ross\AppData\Local\Temp\tmp5i82xkhu\model.ckpt.
INFO:tensorflow:loss = 54.29764, step = 41
INFO:tensorflow:Saving checkpoints for 60 into C:\Users\Ross\AppData\Local\Temp\tmp5i82xkhu\model.ckpt.
INFO:tensorflow:Loss for final step: 47.993824.
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2018-04-22-12:52:41
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from C:\Users\Ross\AppData\Local\Temp\tmp5i82xkhu\model.ckpt-60
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Finished evaluation at 2018-04-22-12:52:41
INFO:tensorflow:Saving dict for global step 60: accuracy = 0.8881168, average_loss = 0.4243504, global_step = 60, loss = 40.70935
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INF

INFO:tensorflow:Starting evaluation at 2018-04-22-12:52:53
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from C:\Users\Ross\AppData\Local\Temp\tmp6qujvhj5\model.ckpt-80
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Finished evaluation at 2018-04-22-12:52:54
INFO:tensorflow:Saving dict for global step 80: accuracy = 0.9583044, average_loss = 0.18116081, global_step = 80, loss = 17.37936
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from C:\Users\Ross\AppData\Local\Temp\tmp6qujvhj5\model.ckpt-80
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 81 into C:\Users\Ross\AppData\Local\Temp\tmp6qujvhj5\model.ckpt.
INFO:tensorflow:loss = 22.734634, step = 81
INFO:tensorflow:Saving checkpoints for 100 i

In [86]:
show_results(kfcv_evaluations)



	Num Results: 20
>> (inner_arch, train_frac, num_steps) , Accuracy
([10], 0.8, 20.0) , 0.74357194
([10], 0.8, 20.0) , 0.8311327
([10], 0.8, 20.0) , 0.82626826
([10], 0.8, 20.0) , 0.8436414
([10], 0.8, 20.0) , 0.8630994
([100], 0.8, 20.0) , 0.88255733
([100], 0.8, 20.0) , 0.92911744
([100], 0.8, 20.0) , 0.923558
([100], 0.8, 20.0) , 0.9437109
([100], 0.8, 20.0) , 0.9409312
([20, 20], 0.8, 20.0) , 0.7678944
([20, 20], 0.8, 20.0) , 0.85128564
([20, 20], 0.8, 20.0) , 0.8881168
([20, 20], 0.8, 20.0) , 0.90340513
([20, 20], 0.8, 20.0) , 0.923558
([100, 100], 0.8, 20.0) , 0.8881168
([100, 100], 0.8, 20.0) , 0.9381515
([100, 100], 0.8, 20.0) , 0.9471855
([100, 100], 0.8, 20.0) , 0.9583044
([100, 100], 0.8, 20.0) , 0.9589993


## Other

### Deletes temp files created by TensorFlow

In [95]:
# Allows deleting models save folder in windows
tf.summary.FileWriterCache.clear()

In [97]:
import tempfile
import os
from pathlib import Path
import shutil

temp_dir = Path(tempfile.gettempdir())
dir_list = [f for f in temp_dir.glob('tmp*') if f.is_dir()] 
for dir in dir_list:
    shutil.rmtree(dir, ignore_errors=True)
        
        
